In [1]:
import cv2
import ctypes
import os

# 1. 얼굴 인식 모델 (가장 가벼운 Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 2. 웹캠 연결
cap = cv2.VideoCapture(0)

# 속도 최적화를 위한 웹캠 해상도 하향 설정 (선택 사항)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

def hide_windows():
    # Windows + D 효과를 내는 쉘 명령 (가장 빠름)
    ctypes.windll.shell32.ShellExecuteW(None, "open", "shell:::{30805640-F057-11D2-9D73-0000F81EF32E}", None, None, 1)

print("보안 감시 가동 중... (종료하려면 Q를 누르세요)")

while True:
    ret, frame = cap.read()
    if not ret: break

    # 속도 향상을 위한 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 얼굴 감지 (minNeighbors 값을 높이면 오작동은 줄어들지만 감지 속도에 영향을 줍니다)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    if len(faces) > 0:
        print("경고: 얼굴 감지됨! 바탕화면으로 전환합니다.")
        hide_windows()
        # 한 번 실행 후 무한 반복 방지를 위해 감시 잠시 중단 (필요시 조정)
        cv2.waitKey(2000) 

    # 실시간 확인이 필요 없다면 아래 창 출력 부분은 주석 처리 시 속도가 더 빨라집니다.
    # cv2.imshow('Security Monitor', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

보안 감시 가동 중... (종료하려면 Q를 누르세요)
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.
경고: 얼굴 감지됨! 바탕화면으로 전환합니다.


KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time

mp_face_detection = mp.solutions.face_detection
# model_selection=1 은 2m 밖의 먼 거리 얼굴에 최적화된 모델입니다.
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # MediaPipe는 RGB 이미지를 사용합니다.
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.detections:
        print("멀리 있는 인물 감지!")
        pyautogui.hotkey('win', 'd')
        time.sleep(5)

    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()

In [ ]:
import cv2
import numpy as np
import pyautogui
import time
import ctypes

pyautogui.FAILSAFE = False
pyautogui.PAUSE = 0

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# L-shape ROI based on your latest arrow image (normalized x, y)
# Tune these points if camera angle changes.
VERTICAL_ROI_NORM = np.array([
    [0.11, 0.04],
    [0.34, 0.04],
    [0.34, 1.00],
    [0.11, 1.00],
], dtype=np.float32)

HORIZONTAL_ROI_NORM = np.array([
    [0.32, 0.20],
    [0.79, 0.20],
    [0.79, 0.43],
    [0.32, 0.43],
], dtype=np.float32)

CAMERA_INDEX_CANDIDATES = (0, 1, 2)
CAMERA_RESOLUTION_CANDIDATES = ((1280, 720), (960, 540), (640, 480))
DETECT_MIN_SIZE = (12, 12)
DETECT_MIN_NEIGHBORS = 2
DETECT_SCALE_FACTOR = 1.03
UPSCALE_FACTOR = 1.8
UPSCALED_MIN_SIZE = (18, 18)

BACKENDS = []
if hasattr(cv2, "CAP_MSMF"):
    BACKENDS.append(("MSMF", cv2.CAP_MSMF))
if hasattr(cv2, "CAP_DSHOW"):
    BACKENDS.append(("DSHOW", cv2.CAP_DSHOW))
BACKENDS.append(("ANY", cv2.CAP_ANY))

def to_pixels(points_norm, frame_w, frame_h):
    pts = points_norm.copy()
    pts[:, 0] *= frame_w
    pts[:, 1] *= frame_h
    return pts.astype(np.int32)

def get_roi_polygons(frame_w, frame_h):
    return [
        to_pixels(VERTICAL_ROI_NORM, frame_w, frame_h),
        to_pixels(HORIZONTAL_ROI_NORM, frame_w, frame_h),
    ]

def build_roi_mask(frame_h, frame_w, polygons):
    mask = np.zeros((frame_h, frame_w), dtype=np.uint8)
    for poly in polygons:
        cv2.fillPoly(mask, [poly], 255)
    return mask

def box_iou(a, b):
    ax1, ay1, aw, ah = a
    bx1, by1, bw, bh = b
    ax2, ay2 = ax1 + aw, ay1 + ah
    bx2, by2 = bx1 + bw, by1 + bh

    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)

    if inter_x2 <= inter_x1 or inter_y2 <= inter_y1:
        return 0.0

    inter = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
    union = (aw * ah) + (bw * bh) - inter
    if union <= 0:
        return 0.0
    return inter / union

def nms(boxes, iou_threshold=0.25):
    if not boxes:
        return []

    boxes = sorted(boxes, key=lambda b: b[2] * b[3], reverse=True)
    kept = []
    for box in boxes:
        if all(box_iou(box, k) < iou_threshold for k in kept):
            kept.append(box)
    return kept

def test_camera_stream(cap, tries=20, min_success=3):
    success = 0
    for _ in range(tries):
        ok, _ = cap.read()
        if ok:
            success += 1
            if success >= min_success:
                return True
        time.sleep(0.01)
    return False

def open_camera(preferred_index=None):
    index_order = list(CAMERA_INDEX_CANDIDATES)
    if preferred_index in index_order:
        index_order.remove(preferred_index)
        index_order.insert(0, preferred_index)

    for idx in index_order:
        for backend_name, backend in BACKENDS:
            for width, height in CAMERA_RESOLUTION_CANDIDATES:
                cap = cv2.VideoCapture(idx, backend)
                if not cap.isOpened():
                    cap.release()
                    continue

                cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
                cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
                cap.set(cv2.CAP_PROP_FPS, 30)
                if hasattr(cv2, "CAP_PROP_BUFFERSIZE"):
                    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                cap.set(cv2.CAP_PROP_FOURCC, fourcc)

                if test_camera_stream(cap):
                    print(f"[INFO] Camera opened (index={idx}, backend={backend_name}, size={width}x{height})")
                    return cap, idx

                cap.release()
    return None

def minimize_desktop():
    try:
        pyautogui.hotkey("win", "d")
        return True
    except Exception:
        try:
            user32 = ctypes.windll.user32
            key_up = 0x0002
            vk_lwin = 0x5B
            vk_d = 0x44
            user32.keybd_event(vk_lwin, 0, 0, 0)
            user32.keybd_event(vk_d, 0, 0, 0)
            user32.keybd_event(vk_d, 0, key_up, 0)
            user32.keybd_event(vk_lwin, 0, key_up, 0)
            return True
        except Exception:
            return False

def detect_faces(gray, polygons, roi_mask):
    all_boxes = []

    for poly in polygons:
        rx, ry, rw, rh = cv2.boundingRect(poly)
        if rw < 10 or rh < 10:
            continue

        roi_gray = gray[ry:ry + rh, rx:rx + rw]
        roi_mask_patch = roi_mask[ry:ry + rh, rx:rx + rw]
        roi_gray = cv2.bitwise_and(roi_gray, roi_gray, mask=roi_mask_patch)

        faces = face_cascade.detectMultiScale(
            roi_gray,
            scaleFactor=DETECT_SCALE_FACTOR,
            minNeighbors=DETECT_MIN_NEIGHBORS,
            minSize=DETECT_MIN_SIZE,
        )

        for (x, y, w, h) in faces:
            all_boxes.append((x + rx, y + ry, w, h))

        up = cv2.resize(roi_gray, None, fx=UPSCALE_FACTOR, fy=UPSCALE_FACTOR, interpolation=cv2.INTER_LINEAR)
        up_faces = face_cascade.detectMultiScale(
            up,
            scaleFactor=DETECT_SCALE_FACTOR,
            minNeighbors=DETECT_MIN_NEIGHBORS,
            minSize=UPSCALED_MIN_SIZE,
        )

        for (x, y, w, h) in up_faces:
            ox = int(x / UPSCALE_FACTOR) + rx
            oy = int(y / UPSCALE_FACTOR) + ry
            ow = int(w / UPSCALE_FACTOR)
            oh = int(h / UPSCALE_FACTOR)
            all_boxes.append((ox, oy, ow, oh))

    merged = nms(all_boxes, iou_threshold=0.25)
    faces_in_roi = []
    for (x, y, w, h) in merged:
        cx = x + w // 2
        cy = y + h // 2
        for poly in polygons:
            if cv2.pointPolygonTest(poly, (float(cx), float(cy)), False) >= 0:
                faces_in_roi.append((x, y, w, h))
                break

    return faces_in_roi

opened = open_camera()
if opened is None:
    raise RuntimeError("Camera open failed. Close other apps using webcam and run again.")

cap, current_cam_index = opened
print("보안 감시 가동 중... (Q를 누르면 종료)")

last_action_time = 0.0
cooldown_sec = 5
read_fail_count = 0
reconnect_attempts = 0

while True:
    ret, frame = cap.read()

    if not ret:
        read_fail_count += 1
        if read_fail_count % 20 == 0:
            print(f"[WARN] Frame read failed x{read_fail_count}")

        if read_fail_count >= 20:
            reconnect_attempts += 1
            print(f"[WARN] Attempting camera reconnect #{reconnect_attempts}...")
            cap.release()
            reopened = open_camera(preferred_index=current_cam_index)
            if reopened is not None:
                cap, current_cam_index = reopened
                read_fail_count = 0
                print("[INFO] Camera stream recovered")
                continue

            if reconnect_attempts >= 10:
                print("[ERROR] Camera stream could not be recovered. Exiting loop.")
                break

            read_fail_count = 0

        time.sleep(0.05)
        continue

    read_fail_count = 0
    reconnect_attempts = 0

    frame_h, frame_w = frame.shape[:2]
    polygons = get_roi_polygons(frame_w, frame_h)
    roi_mask = build_roi_mask(frame_h, frame_w, polygons)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    faces_in_roi = detect_faces(gray, polygons, roi_mask)

    now = time.time()
    if len(faces_in_roi) > 0 and (now - last_action_time) >= cooldown_sec:
        print(f"!!! Face detected in ROI ({len(faces_in_roi)}) !!!")
        if minimize_desktop():
            print("[ACTION] Win+D executed")
        else:
            print("[ERROR] Failed to execute Win+D")
        last_action_time = now

    overlay = frame.copy()
    for poly in polygons:
        cv2.fillPoly(overlay, [poly], (40, 70, 140))
    frame = cv2.addWeighted(overlay, 0.18, frame, 0.82, 0)

    for poly in polygons:
        cv2.polylines(frame, [poly], True, (255, 220, 0), 2)
    for (x, y, w, h) in faces_in_roi:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Security Monitor", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
